In [ ]:
# default_exp evaluation

# Hierarchical Evaluation 

> Module for Hierarchical Evaluation.

In [ ]:
#export
from typing import Callable, Dict, List, Optional

import numpy as np
import pandas as pd

In [ ]:
#hide
from fastcore.test import test_close, test_fail
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#export
class HierarchicalEvaluation:
    
    def __init__(
            self, 
            evaluators: List[Callable] # functions with arguments `y`, `y_hat` 
        ):
        self.evaluators = evaluators
        
    def evaluate(
            self, 
            Y_h: pd.DataFrame, # Forecasts with columns `['ds']` and models to evaluate.
            Y_test: pd.DataFrame, # True values with columns `['ds', 'y']`
            tags: Dict[str, np.ndarray], # Each key is a level and its value contains tags associated to that level.
            benchmark: Optional[str] = None # If passed, evaluators are scaled by the error of this benchark.
        ):
        drop_cols = ['ds', 'y'] if 'y' in Y_h.columns else ['ds']
        model_names = Y_h.drop(columns=drop_cols, axis=1).columns.to_list()
        fn_names = [fn.__name__ for fn in self.evaluators]
        if benchmark is not None:
            fn_names = [f'{fn_name}-scaled' for fn_name in fn_names]
        tags_ = {'Overall': np.concatenate(list(tags.values()))}
        tags_ = {**tags_, **tags}
        index = pd.MultiIndex.from_product([tags_.keys(), fn_names], names=['level', 'metric'])
        evaluation = pd.DataFrame(columns=model_names, index=index)
        for level, cats in tags_.items():
            Y_h_cats = Y_h.loc[cats]
            y_test_cats = Y_test.loc[cats, 'y'].values
            for i_fn, fn in enumerate(self.evaluators):
                fn_name = fn_names[i_fn]
                for model in model_names:
                    loss = fn(y_test_cats, Y_h_cats[model].values)
                    if benchmark is not None:
                        scale = fn(y_test_cats, Y_h_cats[benchmark].values)
                        if np.isclose(scale, 0., atol=np.finfo(float).eps):
                            scale += np.finfo(float).eps
                            if np.isclose(scale, loss, atol=1e-8):
                                scale = 1.
                        loss /= scale
                    evaluation.loc[(level, fn_name), model] = loss
        return evaluation

In [ ]:
#hide
add_docs(HierarchicalEvaluation, "Evaluate reconciliation methods.",
         evaluate="Evaluate reconciliation methods for distinct levels.")

In [ ]:
show_doc(HierarchicalEvaluation)

In [ ]:
show_doc(HierarchicalEvaluation.evaluate)

In [ ]:
#hide
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace, ERM
from hierarchicalforecast.utils import hierarchize
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# non strictly hierarchical structure
hiers_grouped = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]
# strictly hierarchical structure
hiers_strictly = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'State', 'Region'], 
]

# getting df
hier_grouped_df, S_grouped, tags_grouped = hierarchize(df, hiers_grouped)

#split train/test
hier_grouped_df['y_model'] = hier_grouped_df['y']
# we should be able to recover y using the methods
hier_grouped_df_h = hier_grouped_df.groupby('unique_id').tail(12)
ds_h = hier_grouped_df_h['ds'].unique()
hier_grouped_df = hier_grouped_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_grouped_df['y_model'] += np.random.uniform(-1, 1, len(hier_grouped_df))

#hierachical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    # ERM recovers but needs bigger eps
    ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(hier_grouped_df_h, hier_grouped_df, S_grouped, tags_grouped)

#hide
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)
def rmse(y, y_hat):
    return np.sqrt(mse(y, y_hat))
evaluator = HierarchicalEvaluation([mse, rmse])
evaluator.evaluate(Y_h=reconciled.drop(columns='y'), 
                   Y_test=reconciled[['ds', 'y']], 
                   tags=tags_grouped,
                   benchmark='y_model')